In [34]:
import tensorflow as tf
import numpy as np
import netCDF4
from tensorflow.keras.utils import plot_model


def get_mask_y():
    # Get the mask for the input data
    y_path = '/nobackup/users/schreurs/project_GAN/dataset_aart'
    # The mask is the same for all radar scans, so simply chose a random one to get the mask
    path = y_path + '/RAD_NL25_RAC_MFBS_EM_5min_201901010000.nc'

    with netCDF4.Dataset(path, 'r') as f:
        rain = f['image1_image_data'][:]
        mask = (rain == 65535)
    mask = mask.astype(float)
    mask = np.expand_dims(mask, axis=0)
    mask = np.expand_dims(mask, axis=-1)

    mask = crop_center(mask)
    return mask

def crop_center(img,cropx=350,cropy=384):
    # batch size, sequence, height, width, channels
     # Only change height and width
    _,_, y,x, _ = img.shape
    startx = 20+x//2-(cropx//2)
    starty = 40+y//2-(cropy//2)    
    return img[:,:,starty:starty+cropy,startx:startx+cropx:,]

def model():
    input_seq = tf.keras.Input(shape=(5, 768, 700, 1))

    down = tf.keras.layers.Conv2D(filters=8, kernel_size=(5, 5), strides=(2,2), padding='same')(input_seq)
    down = tf.keras.layers.LeakyReLU()(down)
    
    convLSTM = tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(convLSTM)
    
    convLSTM = tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3), 
                                          strides=(1,1),
                                          padding='same', 
                                          return_sequences=True)(convLSTM)
    
    output = tf.keras.layers.Conv2D(filters=1, kernel_size=(1, 1), 
                                    strides=(1,1), padding='same')(up)
    
    output = tf.keras.layers.Reshape(target_shape=(1,765, 700, 1))(output)
    
    output_with_mask = tf.keras.layers.Multiply()([down, get_mask_y()])
    
    model = tf.keras.Model(inputs=input_seq, outputs=output_with_mask)
    return model

model = model()
model.summary()
#plot_model(model)
model.compile( optimizer='sgd',
    loss='mse',
    metrics=[tf.keras.metrics.MeanSquaredError()])


UnboundLocalError: local variable 'convLSTM' referenced before assignment

In [29]:
from batchcreator import DataGenerator

In [ ]:
class MyModel(tf.keras.Model):

    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(5, activation=tf.nn.softmax)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)